# DATA 410 Lecture 10 - Spring 2022

<font face="Chalkboard" color="darkgreen" size=10> Multivariate Models for Regression</font>

In [2]:
import numpy as np
import pandas as pd
from scipy.linalg import lstsq
from scipy.sparse.linalg import lsmr
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, griddata, LinearNDInterpolator, NearestNDInterpolator
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, train_test_split as tts
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
import matplotlib.pyplot as plt
from matplotlib import pyplot

## Gradient Boosting

Assume you have an regressor $F$ and, for the observation $x_i$ we make the prediction $F(x_i)$. To improve the predictions, we can regard $F$ as a 'weak learner' and therefore train a decision tree (we can call it $h$) where its predictions are $y_i-F(x_i)$. Thus, there are increased chances that the new regressor

$$\large F + h$$ 

is better than the old one, $F.$

In [3]:
# import libraries for creating a neural network
# imports for creating a Neural Network
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from sklearn.metrics import r2_score
from tensorflow.keras.optimizers import Adam # they recently updated Tensorflow
from keras.callbacks import EarlyStopping

In [4]:
# Tricubic Kernel
def Tricubic(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,70/81*(1-d**3)**3)

# Quartic Kernel
def Quartic(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,15/16*(1-d**2)**2)

# Epanechnikov Kernel
def Epanechnikov(x):
  if len(x.shape) == 1:
    x = x.reshape(-1,1)
  d = np.sqrt(np.sum(x**2,axis=1))
  return np.where(d>1,0,3/4*(1-d**2)) 

In [5]:
#Defining the kernel local regression model

def lw_reg(X, y, xnew, kern, tau, intercept):
    # tau is called bandwidth K((x-x[i])/(2*tau))
    n = len(X) # the number of observations
    yest = np.zeros(n)

    if len(y.shape)==1: # here we make column vectors
      y = y.reshape(-1,1)

    if len(X.shape)==1:
      X = X.reshape(-1,1)
    
    if intercept:
      X1 = np.column_stack([np.ones((len(X),1)),X])
    else:
      X1 = X

    w = np.array([kern((X - X[i])/(2*tau)) for i in range(n)]) # here we compute n vectors of weights

    #Looping through all X-points
    for i in range(n):          
        W = np.diag(w[:,i])
        b = np.transpose(X1).dot(W).dot(y)
        A = np.transpose(X1).dot(W).dot(X1)
        #A = A + 0.001*np.eye(X1.shape[1]) # if we want L2 regularization
        #theta = linalg.solve(A, b) # A*theta = b
        beta, res, rnk, s = lstsq(A, b)
        yest[i] = np.dot(X1[i],beta)
    if X.shape[1]==1:
      f = interp1d(X.flatten(),yest,fill_value='extrapolate')
    else:
      f = LinearNDInterpolator(X, yest)
    output = f(xnew) # the output may have NaN's where the data points from xnew are outside the convex hull of X
    if sum(np.isnan(output))>0:
      g = NearestNDInterpolator(X,y.ravel()) 
      # output[np.isnan(output)] = g(X[np.isnan(output)])
      output[np.isnan(output)] = g(xnew[np.isnan(output)])
    return output

In [28]:
def boosted_lwr(X, y, xtest, kern, tau, intercept):
  # we need decision trees
  # for training the boosted method we use X and y
  Fx = lw_reg(X,y,X,kern,tau,intercept) # we need this for training the Decision Tree
  # Now train the Decision Tree on y_i - F(x_i)
  grad_neg = y - Fx #Resdiduals are the negative gradient for MSE Loss function
  tree_model = DecisionTreeRegressor(max_depth=2, random_state=123)
  tree_model.fit(X,grad_neg)
  output = lw_reg(X,y,xtest,kern,tau,intercept) + tree_model.predict(xtest)
  return output 

## Create a "boosted" (in the sense of Gradient Boosting) version of the Locally Weighted Regressor

In [29]:
cars = pd.read_csv('Data/cars.csv')

X = cars[['ENG','CYL','WGT']].values
y = cars['MPG'].values

In [30]:
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
scale = StandardScaler()

In [33]:
mse_blwr = []
mse_lwr = []
mse_rf = []
mse_nn = []

states = [1234, 2345, 3456, 4567, 5678]

total_mse_blwr = []
total_mse_lwr = []
total_mse_rf = []
total_mse_nn = []

# this is the Cross-Validation Loop
for state in states:
  total_mse_blwr = []
  total_mse_lwr = []
  total_mse_rf = []
  total_mse_nn = []

  kf = KFold(n_splits=5,shuffle=True,random_state=state)
  for idxtrain, idxtest in kf.split(X):
    xtrain = X[idxtrain]
    ytrain = y[idxtrain]
    ytest = y[idxtest]
    xtest = X[idxtest]
    xtrain = scale.fit_transform(xtrain)
    xtest = scale.transform(xtest)

    yhat_lwr = lw_reg(xtrain,ytrain, xtest,Tricubic,tau=0.5,intercept=True)
    yhat_blwr = boosted_lwr(xtrain,ytrain, xtest,Tricubic,tau=1,intercept=True)

    model_rf = RandomForestRegressor(n_estimators=100, max_depth=4, random_state = 123)
    model_rf.fit(xtrain,ytrain)
    yhat_rf = model_rf.predict(xtest)

    #model_nn.fit(xtrain,ytrain,validation_split=0.3, epochs=500, batch_size=20, verbose=0, callbacks=[es])
    #yhat_nn = model_nn.predict(xtest)

    mse_lwr.append(mse(ytest,yhat_lwr))
    mse_blwr.append(mse(ytest,yhat_blwr))
    mse_rf.append(mse(ytest,yhat_rf))
    #mse_nn.append(mse(ytest,yhat_nn))
  
  total_mse_lwr.append(np.mean(mse_lwr))
  total_mse_blwr.append(np.mean(mse_blwr))
  total_mse_rf.append(np.mean(mse_rf))
  #total_mse_nn.append(np.mean(mse_nn))

mse_blwr = []
mse_lwr = []
mse_rf = []
mse_nn = []
  
  

print('The Cross-validated Mean Squared Error for LWR is : '+str(np.mean(total_mse_lwr)))
print('The Cross-validated Mean Squared Error for BLWR is : '+str(np.mean(total_mse_blwr)))
print('The Cross-validated Mean Squared Error for RF is : '+str(np.mean(total_mse_rf)))
#print('The Cross-validated Mean Squared Error for NN is : '+str(np.mean(total_mse_nn)))

The Cross-validated Mean Squared Error for LWR is : 17.090059925939794
The Cross-validated Mean Squared Error for BLWR is : 17.64648807322922
The Cross-validated Mean Squared Error for RF is : 17.77482685909969


## Here are the predictions we made for the test data:

In [35]:
from sklearn.metrics import mean_absolute_error


model_nn.fit(xtrain,ytrain,validation_split=0.3, epochs=500, batch_size=20, verbose=0, callbacks=[es])
yhat_nn = model.predict(dat_test[:,:-1])
mae_nn = mean_absolute_error(dat_test[:,-1], yhat_nn)
print("MAE Neural Network = ${:,.2f}".format(1000*mae_nn))

NameError: name 'model_nn' is not defined

In [ ]:
fig, ax = plt.subplots(figsize=(12,9))
ax.set_xlim(3, 9)
ax.set_ylim(0, 51)
ax.scatter(x=df['rooms'], y=df['cmedv'],s=25)
ax.plot(X_test, lm.predict(X_test), color='red',label='Linear Regression')
ax.plot(dat_test[:,0], yhat_nn, color='lightgreen',lw=2.5,label='Neural Network')
ax.plot(dat_test[:,0], model_lowess(dat_train,dat_test,Epanechnikov,0.53), color='orange',lw=2.5,label='Kernel Weighted Regression')
ax.set_xlabel('Number of Rooms',fontsize=16,color='navy')
ax.set_ylabel('House Price (Thousands of Dollars)',fontsize=16,color='navy')
ax.set_title('Boston Housing Prices',fontsize=16,color='purple')
ax.grid(b=True,which='major', color ='grey', linestyle='-', alpha=0.8)
ax.grid(b=True,which='minor', color ='grey', linestyle='--', alpha=0.2)
ax.minorticks_on()
plt.legend()

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=1693)

In [ ]:
mae_lm = []

for idxtrain, idxtest in kf.split(dat):
  X_train = dat[idxtrain,0]
  y_train = dat[idxtrain,1]
  X_test  = dat[idxtest,0]
  y_test = dat[idxtest,1]
  lm.fit(X_train.reshape(-1,1),y_train)
  yhat_lm = lm.predict(X_test.reshape(-1,1))
  mae_lm.append(mean_absolute_error(y_test, yhat_lm))
print("Validated MAE Linear Regression = ${:,.2f}".format(1000*np.mean(mae_lm)))

Validated MAE Linear Regression = $4,447.94


In [ ]:
%%timeit -n 1

mae_lk = []

for idxtrain, idxtest in kf.split(dat):
  dat_test = dat[idxtest,:]
  y_test = dat_test[np.argsort(dat_test[:, 0]),1]
  yhat_lk = model_lowess(dat[idxtrain,:],dat[idxtest,:],Gaussian,0.15)
  mae_lk.append(mean_absolute_error(y_test, yhat_lk))
print("Validated MAE Local Kernel Regression = ${:,.2f}".format(1000*np.mean(mae_lk)))

Validated MAE Local Kernel Regression = $4,090.03
Validated MAE Local Kernel Regression = $4,090.03
Validated MAE Local Kernel Regression = $4,090.03
1 loop, best of 3: 531 ms per loop


In [ ]:
#%%timeit -n 1

mae_nn = []

for idxtrain, idxtest in kf.split(dat):
  X_train = dat[idxtrain,0]
  y_train = dat[idxtrain,1]
  X_test  = dat[idxtest,0]
  y_test = dat[idxtest,1]
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=500)
  model.fit(X_train.reshape(-1,1),y_train,validation_split=0.3, epochs=1000, batch_size=100, verbose=0, callbacks=[es])
  yhat_nn = model.predict(X_test.reshape(-1,1))
  mae_nn.append(mean_absolute_error(y_test, yhat_nn))
print("Validated MAE Neural Network Regression = ${:,.2f}".format(1000*np.mean(mae_nn)))

Epoch 00501: early stopping
Epoch 00506: early stopping
Epoch 00539: early stopping
Epoch 00800: early stopping
Epoch 00672: early stopping
Epoch 00689: early stopping
Epoch 00551: early stopping
Validated MAE Neural Network Regression = $5,108.02


## XGBoost

The method is related to Random Forest

https://towardsdatascience.com/xgboost-python-example-42777d01001e

In [37]:
import xgboost as xgb

In [38]:
model_xgb = xgb.XGBRegressor(objective ='reg:squarederror',n_estimators=100,reg_lambda=20,alpha=1,gamma=10,max_depth=3)

In [41]:
%%timeit -n 1

mae_xgb = []

for idxtrain, idxtest in kf.split(X):
  X_train = X[idxtrain]
  y_train = y[idxtrain]
  X_test  = X[idxtest]
  y_test = y[idxtest]
  model_xgb.fit(X_train.reshape(-1,1),y_train)
  yhat_xgb = model_xgb.predict(X_test.reshape(-1,1))
  mae_xgb.append(mean_absolute_error(y_test, yhat_xgb))
print("Validated MAE XGBoost Regression = ${:,.2f}".format(1000*np.mean(mae_xgb)))

XGBoostError: [14:47:07] C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/data/data.cc:592: Check failed: labels_.Size() == num_row_ (313 vs. 939) : Size of labels must equal to number of rows.

In [ ]:
cstring = 'c'*364

## Using Kernel Regression from StatsModels

In [ ]:
hello = 'c'*dat_train_poly.shape[1]

In [ ]:
hello

'ccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccccc'

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
poly = PolynomialFeatures(degree=2)
dat_train_poly = poly.fit_transform(dat_train)

In [ ]:
from statsmodels.nonparametric.kernel_regression import KernelReg


model_KernReg = KernelReg(endog=dat_train[:,-1],exog=dat_train_poly,var_type=hello,ckertype='gaussian')

In [ ]:
yhat_sm_test, y_std = model_KernReg.fit(dat_test[:,:-1])

In [ ]:
mae_sm = mean_absolute_error(dat_test[:,-1], yhat_sm_test)
print("MAE StatsModels Kernel Regression = ${:,.2f}".format(1000*mae_sm))

MAE StatsModels Kernel Regression = $2,854.57
